## **Loca Como Tu Madre**

In [1]:
from IPython import display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
import datetime
import nbformat
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected = True )
import plotly.graph_objs as go
import plotly.offline as py
from matplotlib.backends.backend_pdf import PdfPages
from fpdf import FPDF
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import locale
import plotly.io as pio
from datetime import date
import calendar

In [2]:
# 1) git log --pretty=format:%cI,%h,%an,%ae,%s > /home/claudio/@LCTM/commits_LCTM.csv
# 2) abrir excel e e na aba dados, importar o arquivo gerado pelo código acima


In [3]:
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf-8')

# Importing each csv file and then joinning then, 
# as the files for other months are send by the client we can insert each one at the end of this code

df1 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_07.csv', encoding="ISO-8859-1", sep = ';')
df2 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_08.csv', encoding="ISO-8859-1", sep = ';')
df3 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_09.csv', encoding="ISO-8859-1", sep = ';')
df4 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_10.csv', encoding="ISO-8859-1", sep = ';')
df5 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_11.csv', encoding="ISO-8859-1", sep = ';')
df6 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_12.csv', encoding="ISO-8859-1", sep = ';')
df7 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2023_01.csv', encoding="ISO-8859-1", sep = ';')

In [4]:
transactions = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [5]:
df1.head()

,Cod Venda,NFe,NFCe,Data Venda,Data Recebimento,Nome Cliente,Valor Venda,Valor Recebido,Vr. Desconto,Tx Serviço,Operação,Tipo Operação,Caixa,Vendedor,Operador,Data Cancelamento,Operador Cancelamento,Operador Desconto,Operador Desc Repique,NRC,Comandas,Série NFCE,Série NFE,Integração,Observação,CO_CLIENTE,Atendente,IC_REVENDA
0,NaN,Código,Produto,Qtde,Vr. Total,Vr. Unitário,Observação,Hora Lançamento,Computador,Operação,Atendente,CO_VENDA,IC_ITEM_CANCELADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,981201.0,NaN,6490,30/07/2022 20:02:56,30/07/2022 22:58:00,NaN,209,209,0,21,1,LOCA,50901,JESSICA,CAIXA,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1701.0,NaN
2,NaN,421,BURRATA COM JAMON E VERDE,1,49,49,NaN,30/07/2022 20:03:12,SERVIDOR,1,JESSICA,981201,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,41,PORÇAO NACHOS,1,15,15,NaN,30/07/2022 20:17:28,SERVIDOR,1,JESSICA,981201,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,97,JARRA SANGRIA LOCA SUMMER,1,55,55,NaN,30/07/2022 20:22:20,SERVIDOR,1,GABRIEL,981201,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [7]:
del transactions['Vr. Desconto']
del transactions['Valor Venda']
del transactions['Tx Serviço']
del transactions["Caixa"]
del transactions['Operador']
del transactions['Data Cancelamento']
del transactions['Operador Cancelamento']
del transactions['Operador Desconto']
del transactions['Operador Desc Repique']
del transactions['NRC']
del transactions['Comandas']
del transactions['Série NFCE']
del transactions['Série NFE']
del transactions['Integração']
del transactions['Observação']
del transactions['CO_CLIENTE']
del transactions['Atendente']
del transactions['IC_REVENDA']

In [8]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46472 entries, 0 to 46471
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Cod Venda         4831 non-null   float64
 1   NFe               41642 non-null  object 
 2   NFCe              46470 non-null  object 
 3   Data Venda        46472 non-null  object 
 4   Data Recebimento  46472 non-null  object 
 5   Nome Cliente      41650 non-null  object 
 6   Valor Recebido    46472 non-null  object 
 7   Operação          46470 non-null  object 
 8   Tipo Operação     46472 non-null  object 
 9   Vendedor          4830 non-null   object 
dtypes: float64(1), object(9)
memory usage: 3.5+ MB


In [9]:
transactions.head(10)

,Cod Venda,NFe,NFCe,Data Venda,Data Recebimento,Nome Cliente,Valor Recebido,Operação,Tipo Operação,Vendedor
0,NaN,Código,Produto,Qtde,Vr. Total,Vr. Unitário,Hora Lançamento,Atendente,CO_VENDA,NaN
1,981201.0,NaN,6490,30/07/2022 20:02:56,30/07/2022 22:58:00,NaN,209,1,LOCA,JESSICA
2,NaN,421,BURRATA COM JAMON E VERDE,1,49,49,30/07/2022 20:03:12,JESSICA,981201,NaN
3,NaN,41,PORÇAO NACHOS,1,15,15,30/07/2022 20:17:28,JESSICA,981201,NaN
4,NaN,97,JARRA SANGRIA LOCA SUMMER,1,55,55,30/07/2022 20:22:20,GABRIEL,981201,NaN
5,NaN,406,LULAS EMPANADAS,1,38,38,30/07/2022 21:22:51,JESSICA,981201,NaN
6,NaN,2,RISOTINHO DE CAMARÃO,1,52,52,30/07/2022 22:26:07,JESSICA,981201,NaN
7,981101.0,NaN,6489,30/07/2022 21:30:34,30/07/2022 22:23:41,NaN,"6,5",2,LOCA,JESSICA
8,NaN,111,AGUA SEM GAS,1,"6,5","6,5",30/07/2022 21:30:34,JESSICA,981101,NaN
9,981001.0,NaN,6488,30/07/2022 21:36:52,30/07/2022 21:37:14,NaN,36,3,LOCA,MARCIO


In [10]:
# renomeei a coluna 'Tipo Operação' para 'venda'.
transactions.rename(columns = {'Tipo Operação':'venda'}, inplace = True)

In [11]:
# Pandas Drop Rows Based on Column Value
# https://sparkbyexamples.com/pandas/pandas-delete-rows-based-on-column-value/
transactions.drop(transactions[transactions['venda'] == 'CO_VENDA'].index, inplace = True)

In [12]:
# renomeei a coluna 'Cod Venda' para 'prim_col'.
transactions.rename(columns = {'Cod Venda':'prim_col'}, inplace = True)

In [13]:
# If 'prim_col' value is not empty, set the value of column 'venda' to the value of column 'prim_col', else value of column 'venda' stays the same
transactions['venda'] = transactions.apply(lambda row: row['prim_col'] if row['venda'] == 'LOCA' else row['venda'], axis=1)

# If 'prim_col' value is not empty, set the value of column 'venda' to the value of column 'prim_col', else value of column 'venda' stays the same
transactions['venda'] = transactions.apply(lambda row: row['prim_col'] if row['venda'] == 'Balcao' else row['venda'], axis=1)

# Vamos analisar cada parte dessa linha:
#transactions['venda']: Estamos nos referindo à coluna 'venda' do DataFrame transactions.
# Essa coluna será atualizada com base no resultado da operação que será realizada a seguir.
#transactions.apply(...): O método apply() é usado para aplicar uma função a todas as linhas 
# (ou colunas) de um DataFrame. Neste caso, a função está sendo aplicada às linhas, conforme
# especificado pelo argumento axis=1.
#lambda row: ...: Estamos usando uma função lambda, que é uma função anônima e de uso único.
# Essa função aceita um argumento chamado row, que representa uma linha do DataFrame transactions.
#row['prim_col'] if row['venda'] == 'LOCA' else row['venda']: Esta é a lógica da função lambda.
# Estamos usando uma expressão condicional para verificar se o valor da coluna 'venda' na
# linha atual (row['venda']) é igual a 'LOCA'. Se for verdade, o valor da coluna 'prim_col'
# na mesma linha (row['prim_col']) será usado como resultado. Caso contrário, o próprio valor da coluna 'venda' será mantido.

In [14]:
# Função para verificar se o valor é uma string e contém apenas letras, testando se o código da celula anterior funcionou
def is_string_with_letters(value):
    return isinstance(value, str) and value.isalpha()

# Aplicando o filtro para manter apenas as linhas onde o valor da coluna 'venda' é uma string e contém apenas letras
filtered_transactions = transactions[transactions['venda'].apply(is_string_with_letters)]

# filtered_transactions

In [15]:
# Função para verificar se o valor é uma data
def is_date(value):
    try:
        pd.to_datetime(value)
        return True
    except ValueError:
        return False

# Função para aplicar à coluna
def copy_date_or_set_nd(value):
    return value if is_date(value) else 'nd'

In [16]:
# Criar a nova coluna 'data_venda' aplicando a função copy_date_or_set_nd
transactions['data_venda'] = transactions['Data Venda'].apply(copy_date_or_set_nd)

In [17]:
# Criar a nova coluna 'data_recebimento' aplicando a função copy_date_or_set_nd
transactions['data_recebimento'] = transactions['Data Recebimento'].apply(copy_date_or_set_nd)

In [18]:
# Convert the "venda" variable from object type to integer type
transactions['venda'] = np.int64(transactions['venda'])

In [19]:
# Agrupar o DataFrame pela coluna 'venda'
grouped = transactions.groupby('venda')

# Para cada grupo, encontrar a primeira data na coluna "data_venda" e atualizar todas as linhas do grupo
for group_name, group_df in grouped:
    first_date = group_df['data_venda'].apply(is_date).idxmax()
    matching_date = transactions.loc[first_date, 'data_venda']
    
    transactions.loc[transactions['venda'] == group_name, 'data_venda'] = matching_date

In [20]:
# Agrupar o DataFrame pela coluna 'venda'
grouped = transactions.groupby('venda')

# Para cada grupo, encontrar a primeira data na coluna "data_recebimento" e atualizar todas as linhas do grupo
for group_name, group_df in grouped:
    first_date_recebimento = group_df['data_recebimento'].apply(is_date).idxmax()
    matching_date = transactions.loc[first_date_recebimento, 'data_recebimento']
    
    transactions.loc[transactions['venda'] == group_name, 'data_recebimento'] = matching_date

In [21]:
# Agrupando as linhas com base na coluna 'venda'
grouped_transactions = transactions.groupby('venda')

# Preenchendo os valores nulos na coluna 'Vendedor' com o primeiro valor não nulo encontrado dentro do grupo
transactions['Vendedor'] = grouped_transactions['Vendedor'].transform(lambda x: x.ffill().bfill())


In [22]:
# Deletando a primeira coluna, pois não será mais utilizada
del transactions['prim_col']

In [23]:
# Deletando linhas com valor NaN na coluna 'NFe'
transactions = transactions.dropna(subset=['NFe'])

In [24]:
# Renomeando várias colunas
transactions = transactions.rename(columns={
    'NFe': 'cod_produto',
    'NFCe': 'produto',
    'Data Venda': 'quantidade',
    'Data Recebimento' : 'valor_unit',
    'Nome Cliente': 'total_item',
    'Valor Recebido': 'data_lanc',
    'Operação': 'atendente',
    'Vendedor': 'vendedor'
})

In [25]:
# Aplicando o filtro para marcar as linhas onde o valor da coluna 'quantidade' é igual a uma data
filtered_transactions_quantidade= transactions[transactions['quantidade'].apply(is_date)]
# depois quero deletar essa linha filtrada
transactions.drop(filtered_transactions_quantidade.index, inplace=True)

## Mostrando se houveram eventos Corporativos

In [26]:
# Detectando Eventos Corporativos, nesse tipo de registro teremos um valor para a variável atendente = NaN
linhas_com_valores_em_branco = transactions[transactions.isnull().any(axis=1)]
linhas_com_valores_em_branco

,cod_produto,produto,quantidade,valor_unit,total_item,data_lanc,atendente,venda,vendedor,data_venda,data_recebimento
305,245,EVENTO CORPORATIVO,1,3400,3400,01/07/2022 13:36:21,NaN,976201,NaN,01/07/2022 13:36:21,01/07/2022 13:36:21


In [27]:
# depois quero deletar essa linha filtrada de eventos corporativos
transactions.drop(linhas_com_valores_em_branco.index, inplace=True)

In [28]:
# Convert coluna "quantidade" variable from object type to integer type
transactions['quantidade'] = np.int64(transactions['quantidade'])

In [29]:
# change the decimal separator of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")
transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [30]:
# change the decimal separator of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")
transactions['total_item'] = transactions['total_item'].astype(float)

In [31]:
transactions['data_lanc'] = pd.to_datetime(transactions['data_lanc'], format='%d/%m/%Y %H:%M:%S')

In [32]:
transactions['data_venda'] = pd.to_datetime(transactions['data_venda'], format='%d/%m/%Y %H:%M:%S')

In [33]:
transactions['data_recebimento'] = pd.to_datetime(transactions['data_recebimento'], format='%d/%m/%Y %H:%M:%S')

## Criação de colunas para facilitar a análise

In [34]:
# Creating a horario column that is a copy of data_hora and coverting it to datetime and extracting only the day;
transactions['data'] = (transactions['data_lanc'])
transactions['data'] = transactions['data'].dt.date

In [35]:
# add hora and minuto column; using class attributes .hour and .minute

transactions['hora'] = transactions.data_lanc.dt.hour
transactions['minuto'] = transactions.data_lanc.dt.minute

In [36]:
transactions['hora_minuto']=pd.to_datetime(transactions['data_lanc'])

transactions['hora_minuto'] = transactions['hora_minuto'].dt.time

In [39]:
# ### Extracting only the Date from 'data_lanc' column as Datetime format
transactions['data_normal'] = transactions['data_lanc'].dt.normalize()

In [40]:
# Definindo a função time_of_day
def time_of_day(hora):
    """
    Determina se a  compra foi feita pela manhã, tarde ou noite.
    """
    
    if hora < 12:
        
        return 'manhã'
    
    elif hora < 17:
        
        return 'tarde'
    
    else:
        
        return 'noite'
    
transactions['periodo_do_dia'] = transactions.hora.apply(time_of_day)

In [41]:
# Add a day of week variable and a second classifier of weekday or weekend
transactions['dia_da_semana'] = [calendar.day_name[i.weekday()] for i in transactions['data_lanc']]

In [42]:
# Add a tipo_de_dia variable (FS= final de semana or DU=  dia útil)
transactions['tipo_de_dia'] = ['FS' if (i == 'domingo') | (i == 'sábado') else 'DU' for i in 
                                                                            transactions['dia_da_semana']]

## Verificando divergência dos dados

In [44]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [45]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)

In [46]:
#### Creating a Column diference
transactions['diference']=(transactions['total_item'])-(transactions['total_item_verif'])

In [47]:
# continuando o item anterior
resumo = transactions.loc[transactions["confirm"] == True, ["total_item", "total_item_verif", 'diference']].sum()

print (resumo)

total_item          0.0
total_item_verif    0.0
diference           0.0
dtype: float64


Não encontramos nenhuma divergência entre os valores ao efetuarmos a multiplicação do valor de venda do item por sua quantidade, e compararmos com o valor expresso na coluna "total_item", não encontramos nenhuma divergência, o valor dessa diferença é zero.